# A2C - CartPole
Implementation of the A2C RL Algorithm for the OpenAI's Gym environment CartPole-V1 (not in parallel).

In [1]:
%load_ext tensorboard
# Import libraries
import numpy as np
import matplotlib.pyplot as pd
from datetime import datetime

import tensorflow as tf
import tensorflow_probability as tfp

import gym

In [19]:
# Value Fuction Estimator
class Critic(tf.keras.layers.Layer):
    
    def __init__(self):
        super(Critic, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units=128, input_shape=[8,], activation='relu')
        
        self.Dropout = tf.keras.layers.Dropout(rate=0.2)

        self.fc2 = tf.keras.layers.Dense(units=64, activation='relu')
        self.out = tf.keras.layers.Dense(units=1, activation=None)
    
    def call(self, x):
        x = self.fc1(x)
        x = self.Dropout(x, training=True)
        x = self.fc2(x)
        x = self.out(x)
        return x

In [20]:
# Action Value Fuction Estimator (q-network)
class Actor(tf.keras.layers.Layer):
    
    def __init__(self):
        super(Actor, self).__init__()
        
        # 64(share) -> 64(share) -> 32 -> 32 -> mu(tanh) [-1,1]
        # 64(share) -> 64(share) -> 32 -> 32 -> sigma(sigmoid) [0,1]
        self.sharedFC1 = tf.keras.layers.Dense(units=64, input_shape=[8,], activation='relu')
        self.sharedFC2 = tf.keras.layers.Dense(units=64, activation='relu')
        
        self.sharedBatchNorm = tf.keras.layers.BatchNormalization()
        
        self.muFC1 = tf.keras.layers.Dense(units=32, activation='relu')
        self.muFC2 = tf.keras.layers.Dense(units=32, activation='relu')
        
        self.sigmaFC1 = tf.keras.layers.Dense(units=32, activation='relu')
        self.sigmaFC2 = tf.keras.layers.Dense(units=32, activation='relu')
        
        
        self.mu_out = tf.keras.layers.Dense(units=2, activation='tanh')
        self.sigma_out = tf.keras.layers.Dense(units=2, activation='sigmoid')
    
    def call(self, x):
        x = tf.convert_to_tensor(x)
        x = self.sharedFC1(x)
        x = self.sharedFC2(x)
        
        x = self.sharedBatchNorm(x, training=True)
        
        mu = self.muFC1(x)
        mu = self.muFC2(mu)
        mu = self.mu_out(mu)
        
        sigma = self.sigmaFC1(x)
        sigma = self.sigmaFC2(sigma)
        sigma = self.sigma_out(sigma)     
        
        return mu, sigma

In [21]:
# Prepare Tensorboard
!rm -rf ./logs/
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
#%tensorboard --logdir logs/
tf.keras.backend.clear_session()
# Initialize cart pole environment
env = gym.make('LunarLanderContinuous-v2')
# Initialize model, loss and optimizer
actor = Actor()
critic = Critic()
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
mse = tf.keras.losses.MSE
weighted_sparse_ce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# Initialize replay memory
observations = []
# Set hyperparameters
discount = 0.95
max_time_steps = 500
num_episodes = 10000

# Store losses temporary
actor_losses = []
critic_losses = []
accum_reward = 0.

step = 0
# Run for agent and environment for num_episodes
for i_episode in range(num_episodes):
    state = env.reset()
    #breakpoint()
    
    # Agent has 500 trials at max, if it does not fail beforehand
    for t in range(max_time_steps):
        env.render()
        # Compute action
        state = np.reshape(state, [1,8])
        mu, sigma = actor(state)
        
        # sample two values from normal distribution
        mainEngineAction = tf.random.normal((1,), mean=mu[0,0], stddev=sigma[0,0])
        sideEngineAction = tf.random.normal((1,), mean=mu[0,1], stddev=sigma[0,1])
        action = tf.concat([mainEngineAction, sideEngineAction], 0)
        #        mainEngineAction = np.reshape(action, (2,))
        # Execute action and store action, state and reward
        next_state, reward, done, info = env.step(action)
        observations.append((state, action, reward))
        state = next_state
        accum_reward += reward
        # Interrupt the trial if the agent fails
        if done:
            break
        step += 1
        
    print(f"Episode {i_episode + 1} of {num_episodes} finished after {t+1} timesteps")
        
    # Initialize variable for the estimated return
    estimated_return = 0 if done else critic(next_state)
    
    # Iterate over taken actions and observed states and rewards
    observations.reverse()
    for state, action, reward in observations:
        # Compute estimated return
        estimated_return = discount * estimated_return + reward
        # Compute state value
        state_v = critic(state)
    
        # Compute gradients for the actor (policy gradient)
        # Maximize the estimated return
        with tf.GradientTape() as actor_tape:
            mu, sigma = actor(state)
            advantages = estimated_return - int(state_v)
            advantages = tf.cast([[advantages]], tf.float32)
            action_distribution = tfp.distributions.Normal(loc=mu, scale=sigma)
            logprob = action_distribution.log_prob(action)
            #breakpoint()
            actor_loss = logprob * advantages
            #breakpoint()
            # Compute the actor loss (log part of the policy gradient)
            # Compute gradient with respect to the parameters of the actor            
            policy_gradients = actor_tape.gradient(actor_loss, actor.trainable_variables)

        # Compute gradients for the critic
        # minimize MSE for the state value function
        with tf.GradientTape() as critic_tape:
            state_v = critic(state)
            # Compute the loss
            critic_loss = mse(estimated_return, state_v)
            # Compute the gradient
            critic_gradients = critic_tape.gradient(critic_loss, critic.trainable_variables)
            #breakpoint()
            # Accumulate gradients
            #critic_gradients.append(gradients)
            
        # Apply gradients.
        actor_optimizer.apply_gradients(zip(policy_gradients, actor.trainable_variables))
        critic_optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))
        actor_losses.append(actor_loss)
        critic_losses.append(critic_loss)

    observations = []

    # Store summary statistics
    with train_summary_writer.as_default():
        tf.summary.scalar('policy loss', tf.reduce_mean(actor_losses), step=step)
        
        # Store summary statistics
        tf.summary.scalar('critic loss', tf.reduce_mean(critic_losses), step=step)
        
        # Critic
        tf.summary.scalar('V(s)', state_v[0,0], step=step)
        
        # Actor
        tf.summary.scalar('mu0', mu[0,0], step=step)
        tf.summary.scalar('sigma0', sigma[0,0], step=step)
        tf.summary.scalar('mu1', mu[0,1], step=step)
        tf.summary.scalar('sigma1', sigma[0,1], step=step)
    accum_reward = 0.

env.close()

/Users/Alex/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Episode 1 of 10000 finished after 67 timesteps
[[-0.59924716  0.10465702 -1.1712816  -1.2105403   0.5148871  -0.63090223
   0.          1.        ]]
[[-0.58724046  0.13215138 -1.1468297  -1.2334796   0.5465536  -0.20530987
   0.          1.        ]]
[[-0.57569146  0.15997218 -1.1686605  -1.2804921   0.55675066  0.15064433
   0.          1.        ]]
[[-0.56406987  0.18872324 -1.1686642  -1.2538213   0.5492184   0.15064538
   0.          0.        ]]
[[-0.55244845  0.21687491 -1.1549531  -1.2418498   0.5416862   0.1510163
   0.          0.        ]]
[[-0.5409647   0.2447578  -1.1127968  -1.2502949   0.53413534  0.16019846
   0.          0.        ]]
[[-0.52990675  0.27282795 -1.1069416  -1.2216555   0.52612543  0.13244289
   0.          0.        ]]
[[-0.5188955   0.30026507 -1.1069443  -1.1949856   0.5195033   0.13244328
   0.          0.        ]]
[[-0.5078845   0.32710263 -1.0891263  -1.1785682   0.51288116  0.14018273
   0.          0.        ]]
[[-0.4970553   0.35356858 -1.0891293

[[ 0.08895817  0.61868536  0.4958149  -1.1442745  -0.9435872  -0.96138227
   0.          0.        ]]
[[ 0.08429613  0.64385164  0.4916424  -1.1134293  -0.8955181  -0.92660445
   0.          0.        ]]
[[ 0.07968245  0.6683657   0.48605752 -1.0823761  -0.84918797 -0.8872329
   0.          0.        ]]
[[ 0.07512684  0.6922242   0.47777995 -1.0501955  -0.8048264  -0.8344779
   0.          0.        ]]
[[ 0.07064895  0.71540684  0.48427123 -1.0273175  -0.76310253 -0.87330544
   0.          0.        ]]
[[ 0.06613312  0.7380747   0.47779194 -0.996539   -0.7194373  -0.83355
   0.          0.        ]]
[[ 0.06167908  0.7600904   0.45491427 -0.9849568  -0.6777599  -0.7900114
   0.          0.        ]]
[[ 0.05744705  0.7818858   0.4506579  -0.9560769  -0.63825935 -0.76654387
   0.          0.        ]]
[[ 0.05325718  0.80306053  0.41879544 -0.9646113  -0.5999322  -0.7231864
   0.          0.        ]]
[[ 0.04937563  0.8244631   0.37939665 -0.98275435 -0.56377286 -0.6914018
   0.          0

[[ 0.10103884  1.3308991   0.87084997 -0.39238504 -1.4534156  -1.1801037
   0.          0.        ]]
[[ 0.09241962  1.3388413   0.81899536 -0.37703323 -1.3944106  -1.1729805
   0.          0.        ]]
[[ 0.08435269  1.3464526   0.81706583 -0.34517843 -1.3357618  -1.133508
   0.          0.        ]]
[[ 0.07633314  1.3533883   0.76656175 -0.32380596 -1.2790865  -1.1404575
   0.          0.        ]]
[[ 0.06885099  1.3598522   0.76694244 -0.2970382  -1.2220637  -1.1407127
   0.          0.        ]]
[[ 0.06139603  1.3657308   0.7047419  -0.27217144 -1.1650281  -1.0926071
   0.          0.        ]]
[[ 0.05458222  1.3711022   0.6327306  -0.26270717 -1.1103977  -1.1014344
   0.          0.        ]]
[[ 0.0485177   1.3762751   0.63566726 -0.2393114  -1.0553261  -1.1292017
   0.          0.        ]]
[[ 0.04245825  1.3809266   0.6322876  -0.20874855 -0.9988662  -1.097597
   0.          0.        ]]
[[ 0.03645029  1.3849362   0.5932946  -0.19567363 -0.9439865  -1.0521241
   0.          0.   

[[-0.00442677  0.8904887   0.2003295  -0.9358825  -0.58480376 -0.7560185
   0.          0.        ]]
[[-0.00610638  0.9112391   0.16461553 -0.9586058  -0.5470029  -0.7079726
   0.          0.        ]]
[[-0.00744267  0.932537    0.16469009 -0.9318466  -0.51160437 -0.7080331
   0.          0.        ]]
[[-0.00877352  0.9532494   0.15891129 -0.9031418  -0.47620273 -0.6805296
   0.          0.        ]]
[[-0.01005335  0.97334164  0.15360747 -0.8750193  -0.44217628 -0.6567392
   0.          0.        ]]
[[-0.01128616  0.9928245   0.15365928 -0.84826916 -0.40933934 -0.6567869
   0.          0.        ]]
[[-0.01251516  1.0117202   0.13083395 -0.8387417  -0.3765     -0.61564803
   0.          0.        ]]
[[-0.01353159  1.0304275   0.10603372 -0.84938025 -0.3457176  -0.6199256
   0.          0.        ]]
[[-0.01429501  1.0493866   0.06697041 -0.86735725 -0.31472412 -0.58072275
   0.          0.        ]]
[[-0.01468401  1.0687724   0.04528465 -0.8851754  -0.2856906  -0.53947604
   0.          

...

[[-0.05914717  1.4089804  -0.70812273 -0.10334517  0.01638795 -0.09306397
   0.          0.        ]]
[[-0.05201893  1.411307   -0.7176112  -0.07666316  0.02104073 -0.05508048
   0.          0.        ]]
[[-0.04481506  1.4130329  -0.7277799  -0.05044342  0.02379449 -0.01424525
   0.          0.        ]]
[[-0.03753004  1.4141681  -0.7364973  -0.02407327  0.02450669  0.0207641
   0.          0.        ]]
[[-0.03017559  1.4147093  -0.746242    0.00254869  0.02346859  0.05987571
   0.          0.        ]]
[[-0.02274342  1.4146509  -0.75635636  0.02894675  0.02047508  0.10054643
   0.          0.        ]]
[[-0.01523066  1.4139982  -0.764007    0.05561821  0.01544821  0.13139027
   0.          0.        ]]
[[-0.00765705  1.4127457  -0.775585    0.08110724  0.00887933  0.17568168
   0.          0.        ]]
Episode 42 of 10000 finished after 56 timesteps
[[ 0.17247066  0.04395569  0.30740872 -1.9412254  -3.2903857  -2.37016
   0.          0.        ]]
[[ 0.16820058  0.08778306  0.3167128  

[[-0.10369787  0.8397929  -0.11676289 -1.233414   -1.0422839  -1.3176186
   0.          0.        ]]
[[-0.10217305  0.8666993  -0.12037931 -1.202002   -0.97640324 -1.2805089
   0.          0.        ]]
[[-0.10058717  0.89295805 -0.12757541 -1.1688237  -0.91237795 -1.2246084
   0.          0.        ]]
[[-0.09891567  0.9185404  -0.13310024 -1.1375922  -0.85114765 -1.1846755
   0.          0.        ]]
[[-0.09717455  0.9434791  -0.14007851 -1.1063229  -0.791914   -1.1408333
   0.          0.        ]]
[[-0.09535513  0.9677741  -0.14925814 -1.0741206  -0.73487246 -1.0859873
   0.          0.        ]]
[[-0.09344359  0.9914073  -0.1575109  -1.0426052  -0.6805732  -1.0376551
   0.          0.        ]]
[[-0.09145059  1.0143876  -0.16374919 -1.0124289  -0.62869054 -1.002217
   0.          0.        ]]
[[-0.08939447  1.0367365  -0.17110822 -0.9820334  -0.5785797  -0.9624543
   0.          0.        ]]
[[-0.08726845  1.0584488  -0.18189824 -0.9514739  -0.5304571  -0.9105794
   0.          0.  

[[-0.18597022  1.3867338  -0.3313746  -0.5636492  -1.0536203  -1.3099282
   0.          0.        ]]
[[-0.18230781  1.3985691  -0.3361525  -0.531336   -0.9881242  -1.2644659
   0.          0.        ]]
[[-0.17857552  1.409741   -0.3430694  -0.49873695 -0.92490107 -1.2126342
   0.          0.        ]]
[[-0.17475891  1.420246   -0.34786725 -0.46728906 -0.8642695  -1.1732504
   0.          0.        ]]
[[-0.17088003  1.430101   -0.35426444 -0.43559936 -0.80560714 -1.1286485
   0.          0.        ]]
[[-0.16692886  1.4393024  -0.36252123 -0.4031925  -0.74917483 -1.0753512
   0.          0.        ]]
[[-0.16289406  1.4478359  -0.36840707 -0.37243506 -0.6954074  -1.0378525
   0.          0.        ]]
[[-0.15879698  1.4557279  -0.37657577 -0.34097707 -0.6435148  -0.9903113
   0.          0.        ]]
[[-0.15462188  1.4629653  -0.3835297  -0.31129548 -0.5939993  -0.954807
   0.          0.        ]]
[[-0.1503788   1.4695795  -0.38999468 -0.28122315 -0.54625905 -0.91959065
   0.          0. 

[[ 0.32058507  1.4411961   1.1456816  -0.47259876 -1.4728179  -1.6130937
   0.          0.        ]]
[[ 0.30924433  1.4506165   1.1435652  -0.43812096 -1.3921635  -1.5545077
   0.          0.        ]]
[[ 0.29798156  1.4593221   1.1406912  -0.40343532 -1.3144383  -1.4939264
   0.          0.        ]]
[[ 0.28679648  1.4673144   1.0927675  -0.3740746  -1.2397423  -1.442775
   0.          0.        ]]
[[ 0.27613363  1.4747095   1.0883586  -0.34017965 -1.1676036  -1.3864464
   0.          0.        ]]
[[ 0.26555014  1.4814113   1.0850241  -0.30803916 -1.0982814  -1.3443158
   0.          0.        ]]
[[ 0.25503197  1.4874507   1.0789235  -0.27468947 -1.0310658  -1.2891672
   0.          0.        ]]
[[ 0.2445981  1.4928094  1.0731299 -0.2415118 -0.9666076 -1.2361481
   0.         0.       ]]
[[ 0.23423986  1.4974893   1.068297   -0.21002474 -0.90480024 -1.1961448
   0.          0.        ]]
[[ 0.22394666  1.5015191   1.0635736  -0.17819151 -0.8449931  -1.1565716
   0.          0.        ]

KeyboardInterrupt: 

In [16]:
env.observation_space

Box(8,)